In [1]:
from utils.dataset import *
from models.discriminator import *

In [17]:
from models.inverter import *

In [18]:
from models.discriminator import *

In [2]:
train, val, test = get_dataset("ENZYMES")
train_loader = get_dataloader(train, batch_size=64)

In [8]:
for i, data in enumerate(train_loader):
    # print(data.keys())
    X = data['x']
    Y = data['y']
    print(X.shape)
    print(Y.shape)
    print(data['len'])
    break

torch.Size([64, 126, 125])
torch.Size([64, 126, 125])
tensor([44, 46, 20,  8, 42, 88, 20, 36, 26, 18, 38, 23, 30, 47, 29, 19, 18, 33,
        18, 21, 42, 14, 46, 32, 19, 36, 25, 11, 39, 33, 25, 40, 33, 15, 36, 74,
        34, 22, 39, 42, 27, 39, 32, 17, 40, 41, 26, 20, 48, 42, 66, 20, 27, 20,
        18, 42, 36, 19, 42, 28, 23, 14, 19, 12])


In [19]:
I = Inverter(input_dim=512, output_dim=8, hidden_dim=256)
D = NetD(stat_input_dim=128, stat_hidden_dim=64, num_stat=2)

In [32]:
optimizerI = optim.Adam(I.parameters(), lr=1e-3)
lossI = WGAN_ReconLoss(0.1, 'MSE')
optimizerD = optim.Adam(D.parameters(), lr=1e-3, betas=[1e-5 for _ in range(2)])

In [37]:
noise = torch.randn(64, 8).to('cuda')
one = torch.FloatTensor([1])
mone = one * -1

for i, data in enumerate(train_loader):
    X = data['x'].numpy()
    Y = data['y'].numpy()
    for p in D.parameters(): # reset requires_grad
        p.requires_grad = True
    Diters = 10 # number of iterations to train discriminator
    j = 0 # counter for 1, 2, ... Diters
    while j < Diters and i < len(train_loader):
        j += 1
        # weight clipping: clamp parameters to a cube
        for p in D.parameters():
            p.data.clamp_(-0.01, 0.01)
        D.zero_grad()

        # train with real
        print(type(Y))
        input = Y.copy()
            # insert data processing
        errD_real = D(input)
        errD_real.backward(one) # discriminator should assign 1's to true samples

        # train with fake
        input = noise.resize_(64, 1).normal_(0, 1)
            # insert data processing
        fake = D(input)
        errD_fake = D(fake)
        errD_fake.backward(mone) # discriminator should assign -1's to fake samples??

        # compute Wasserstein distance and update parameters
        errD = errD_real - errD_fake
        optimizerD.step()

<class 'numpy.ndarray'>


AttributeError: 'numpy.ndarray' object has no attribute 'degree'